# Code demonstrating exponential reduced rank models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg
import torch

from janelia_core.ml.reduced_rank_models import RRExpModel

## Setup ground truth model

In [3]:
d_in, d_out, d_latent, n_smps = 10, 5, 2, 12000

In [4]:
true_model = RRExpModel(d_in, d_out, d_latent)
true_model.generate_random_model(var_range=[.5, 1])

## Generate data

In [5]:
x = torch.randn([n_smps, d_in])
y = true_model.generate(x)

## Create model we will fit to data

In [6]:
fitted_model = RRExpModel(d_in, d_out, d_latent)
fitted_model.init_weights(y)
#fitted_model.to('cuda')

## Fit model to data

In [20]:
do_prefit = False
adam_params = {'lr': .001, 'betas': [.9, .999]} # betas are for mean and second moment (.9, .999)

# Initially fit all but variances
if do_prefit:
    first_parameters = []
    for param_name, param in fitted_model.named_parameters():
        if param_name not in {'v'}:
            first_parameters.append(param)
    fitted_model.fit(x,y, batch_size=1000, send_size=1000,
                 max_its=10000, update_int=1000, adam_params=adam_params, parameters=first_parameters)

# Now fit everything
fitted_model.fit(x,y, batch_size=10000, send_size=10000,
                 max_its=50000, update_int=1000, adam_params=adam_params)    

0: Elapsed time 0.01558232307434082, vl: 3.4341265625
1000: Elapsed time 5.981644153594971, vl: 3.43988125
2000: Elapsed time 11.889549016952515, vl: 3.1296654296875
3000: Elapsed time 51.6052770614624, vl: 2.980594921875
4000: Elapsed time 79.68609690666199, vl: 2.860013671875
5000: Elapsed time 85.61082363128662, vl: 2.7748107421875
6000: Elapsed time 91.53561305999756, vl: 2.743262890625
7000: Elapsed time 97.5242931842804, vl: 3.844874609375
8000: Elapsed time 106.09387493133545, vl: 2.7128744140625
9000: Elapsed time 112.00358247756958, vl: 2.7011265625
10000: Elapsed time 117.9877233505249, vl: 2.716837109375
11000: Elapsed time 123.9160840511322, vl: 2.72785
12000: Elapsed time 129.79482007026672, vl: 4.128894921875
13000: Elapsed time 135.72058510780334, vl: 2.73646796875
14000: Elapsed time 141.61937355995178, vl: 2.7092826171875
15000: Elapsed time 147.6368465423584, vl: 2.720743359375
16000: Elapsed time 153.5919852256775, vl: 6.11785859375
17000: Elapsed time 161.8570795059

## See fitted model results

In [21]:
%matplotlib qt
true_model.standardize()
fitted_model.standardize()
fitted_model.to('cpu')
true_model.compare_models(true_model, fitted_model, x[0:100,:], [0, 1,2])

Standardize called.
Standardize called.


In [9]:
plt.figure()
plt.plot(true_model.w0[:,0].detach().numpy())
plt.plot(fitted_model.w0[:,0].detach().numpy())

## Look at some input/output curves

In [10]:
[mn, g_out, exp_in, l] = true_model.latent_rep(x)

mn = mn.cpu().detach().numpy()
g_out = g_out.cpu().detach().numpy()
exp_in = exp_in.cpu().detach().numpy()
l = l.cpu().detach().numpy()

In [11]:
n = 2
for i in range(n):
    plt.plot(exp_in[:,i], g_out[:,i], 'o')

In [12]:
np.exp(-3)

0.049787068367863944

In [13]:
true_model.o2

Parameter containing:
tensor([[0.5280],
        [0.1163],
        [0.5151],
        [0.1076],
        [0.3607]], requires_grad=True)

In [14]:
a = list(true_model.parameters())

In [15]:
fitted_model.v

Parameter containing:
tensor([[2.6879],
        [5.0487],
        [1.8011],
        [3.0061],
        [1.4475]], requires_grad=True)